In [1]:
%pylab qt
%load_ext autoreload
%autoreload 2

%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


In [10]:
# Loop through the ggems mhd files and save them to an array
import glob
import fastcat as fc
import numpy as np
import matplotlib.pyplot as plt
import mpl_interactions as pli
from mpl_interactions import ipyplot as iplt
# Zoom the ggems projections
from scipy.ndimage import zoom

ggems_scatter_files = glob.glob(
    'test/out/ggems_1e09_121kVp_*scatter.mhd'
)
# find files that end in scatter but don't have flood in the name

ggems_primary_files = glob.glob(
    'test/out/ggems_1e09_121kVp_*[!scatter].mhd'
)
# ggems_primary_files = [x for x in ggems_primary_files if 'flood' not in x]
# ggems_scatter_files = [x for x in ggems_scatter_files if 'flood' not in x]

ggems_scatter_files.sort()
ggems_primary_files.sort()

ggems_primary_projections = []
ggems_scatter_projections = []

for ggems_scatter_file, ggems_primary_file in zip(ggems_scatter_files, ggems_primary_files):
    ggems_prmiary, b, c = fc.utils.read_mhd(ggems_primary_file)
    ggems_scatter, b, c = fc.utils.read_mhd(ggems_scatter_file)

    ggems_primary_projections.append(ggems_prmiary.squeeze())
    ggems_scatter_projections.append(ggems_scatter.squeeze())

# Load the pickled phantom
pkl_file = 'test/ggems_1e08_121kVp.pkl'
phantom = np.load(pkl_file, allow_pickle=True)

# Convert the phantom angles to degrees
phantom_angles = phantom.angles * 180 / np.pi

# Define the denoising functions
def downsize_block(image, block_size):
    image_size = image.shape[0]
    block_size = int(block_size)
    downsize = image_size//block_size
    image = image[:downsize*block_size, :downsize*block_size]
    image = image.reshape(downsize, block_size, downsize,
                          block_size).mean(axis=(1, 3))
    return image

def denoise_projections(projections):
    projections_zoomed = []

    for projection in projections:
        projection_zoomed_downsized = downsize_block(projection, 64)
        projection_zoomed = zoom(
            projection_zoomed_downsized, 64, order=4, mode='nearest')
        # Adjust the mean of the zoomed image to match the original
        projection_zoomed = projection_zoomed * \
            np.mean(projection) / np.mean(projection_zoomed)
        projections_zoomed.append(projection_zoomed)
    
    return projections_zoomed

# zoom the scatter projections using the zoom fromfunction
ggems_scatter_denoised = denoise_projections(ggems_scatter_projections)

/tmp/ipykernel_7806/2118662454.py:60: RuntimeWarning: invalid value encountered in divide
  projection_zoomed = projection_zoomed * \


In [11]:
plt.figure()
pli.heatmap_slicer(np.arange(ggems_primary_projections[0].shape[0]), np.arange(ggems_primary_projections[0].shape[1]), ggems_primary_projections[1])

(<Figure size 1800x900 with 2 Axes>,
 array([<Axes: title={'center': 'heatmap_0'}, xlabel='X', ylabel='Y'>,
        <Axes: title={'center': 'Horizontal'}>], dtype=object))

In [48]:
plt.figure()
pli.hyperslicer(ggems_primary_projections, axis=0)

AttributeError: 'Controls' object has no attribute 'vbox'

In [18]:
# Get the simulation parameters
simulation_parameters = phantom.simulation_parameters
detector_material = simulation_parameters['detector_material']
spectrum = simulation_parameters['spectrum']

# Load the spectrum as a fastcat spectrum object
s = fc.Spectrum()
s.load(spectrum_file=spectrum)

det = fc.Detector(s,'CsI-784-micrometer')
angles = phantom.angles

# return the projections
phantom.return_projs(det,s,angles,mgy=0,
                    ASG=False, scat_on=False,
                    det_on=False,
                    filter_on=False,
                    return_intensity=True)# Get primary fastcat projections

[2023-12-15 15:25:00,983] {ggems_simulate.py:470} INFO -     Interp scatter 512 to 512 pixels
[2023-12-15 15:25:00,984] {ggems_simulate.py:470} INFO -     Interp scatter 512 to 512 pixels


[2023-12-15 15:25:01,836] {ggems_simulate.py:573} INFO - Running Simulations
[2023-12-15 15:25:01,837] {ggems_simulate.py:600} INFO -     Simulating 10 keV
[2023-12-15 15:25:02,627] {ggems_simulate.py:600} INFO -     Simulating 20 keV
[2023-12-15 15:25:03,432] {ggems_simulate.py:600} INFO -     Simulating 30 keV
[2023-12-15 15:25:04,245] {ggems_simulate.py:600} INFO -     Simulating 40 keV
[2023-12-15 15:25:05,079] {ggems_simulate.py:600} INFO -     Simulating 50 keV
[2023-12-15 15:25:05,899] {ggems_simulate.py:600} INFO -     Simulating 60 keV
[2023-12-15 15:25:06,731] {ggems_simulate.py:600} INFO -     Simulating 70 keV
[2023-12-15 15:25:07,552] {ggems_simulate.py:600} INFO -     Simulating 80 keV
[2023-12-15 15:25:08,358] {ggems_simulate.py:600} INFO -     Simulating 90 keV
[2023-12-15 15:25:09,186] {ggems_simulate.py:600} INFO -     Simulating 100 keV
[2023-12-15 15:25:10,000] {ggems_simulate.py:815} INFO - Weighting simulations
[2023-12-15 15:25:10,000] {ggems_simulate.py:898} INF

In [21]:
plt.figure()
pli.hyperslicer(phantom.intensity, axis=0, cmap='gray')

ggems_primary_only = np.array(ggems_primary_projections) - np.array(ggems_scatter_projections)
# Set the mean and standard deviation of the ggems primary only projections to match the fastcat projections
def normalize_mean_and_std(array1,array2):
    array1 = array1 - np.mean(array1)
    array1 = array1 * np.std(array2) / np.std(array1)
    array1 = array1 + np.mean(array2)
    return array1

phantom_projections = normalize_mean_and_std(phantom.intensity, ggems_primary_only)


In [22]:

# Set up a shallow neural network to map the fastcat projections to the ggems_primary_only projections using sklearn
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    phantom.intensity[0].flatten()[::4], ggems_primary_only[0].flatten()[::4], test_size=0.2, random_state=42)

# Scale the data
scaler = StandardScaler()
scaler.fit(X_train.reshape(-1, 1))
X_train_scaled = scaler.transform(X_train.reshape(-1, 1))
X_test_scaled = scaler.transform(X_test.reshape(-1, 1))

# # Define the neural network
mlp = MLPRegressor(hidden_layer_sizes=(10,10,10),
                   activation='relu',
                   solver='adam',
                   max_iter=100,
                   verbose=True,
                   random_state=42)

# Use an svm instead of a neural network
# from sklearn.svm import SVR
# mlp = SVR(kernel='rbf', C=100, gamma=0.1, epsilon=.1)

# Train the neural network
mlp.fit(X_train_scaled, y_train)

# Predict the test data
predictions = mlp.predict(X_test_scaled)

# Plot the predictions vs the actual data
plt.figure()
plt.scatter(y_test, predictions)
plt.xlabel('True Values')
plt.ylabel('Predictions')

# Plot the residuals
plt.figure()
plt.scatter(y_test, y_test - predictions)
plt.xlabel('True Values')
plt.ylabel('Residuals')

# Calculate the mean squared error
print('Mean squared error: ', mean_squared_error(y_test, predictions))

Iteration 1, loss = 1400002.77787345
Iteration 2, loss = 1241739.66677797
Iteration 3, loss = 615787.99761951
Iteration 4, loss = 81789.76715347
Iteration 5, loss = 3033.68538660
Iteration 6, loss = 1408.08990908
Iteration 7, loss = 1358.92545827
Iteration 8, loss = 1313.81691132
Iteration 9, loss = 1269.50311699
Iteration 10, loss = 1228.44613904
Iteration 11, loss = 1193.59748598
Iteration 12, loss = 1166.57180355
Iteration 13, loss = 1146.27250328
Iteration 14, loss = 1130.54984575
Iteration 15, loss = 1114.68371728
Iteration 16, loss = 1098.43293357
Iteration 17, loss = 1081.56929355
Iteration 18, loss = 1064.19577203
Iteration 19, loss = 1046.37553735
Iteration 20, loss = 1029.43494841
Iteration 21, loss = 1012.43633144
Iteration 22, loss = 996.01427700
Iteration 23, loss = 980.05408660
Iteration 24, loss = 965.47676372
Iteration 25, loss = 951.88727411
Iteration 26, loss = 940.67762853
Iteration 27, loss = 930.92279791
Iteration 28, loss = 923.57547861
Iteration 29, loss = 917.24

In [23]:
# Use the neural network to predict the ggems primary only projections from the fastcat projections
ggems_primary_only_nn = []
for projection in phantom.intensity:
    projection_scaled = scaler.transform(projection.flatten().reshape(-1, 1))
    projection_nn = mlp.predict(projection_scaled)
    projection_nn = projection_nn.reshape(projection.shape)
    ggems_primary_only_nn.append(projection_nn)

# scale the ggems neural network projections to match the fastcat projections
ggems_fastcat_nn = normalize_mean_and_std(np.array(ggems_primary_only_nn), np.array(ggems_primary_only))

In [30]:
# Plot the ggems primary only projections
plt.figure()
cl = pli.hyperslicer(ggems_primary_only, cmap='gray')

In [29]:
perc_diff_primary = (np.array(ggems_primary_only)-np.array(ggems_fastcat_nn[-1::-1]))/np.max(ggems_primary_only)*100

plt.figure()
ct = pli.hyperslicer(np.array(perc_diff_primary), cmap='bwr',
                        vmax=5,vmin=-5)

ValueError: operands could not be broadcast together with shapes (16,512,512) (15,512,512) 

In [16]:
phantom_projections = ggems_fastcat_nn

# Downsize the perc_diff using the downsize_block function
perc_diff = (np.array(ggems_scatter_denoised)-np.array(ggems_scatter_projections))/np.max(ggems_scatter_projections)*100
perc_diff_small = []
for i in range(len(perc_diff)):
    perc_diff_small.append(downsize_block(perc_diff[i], 16))

# Plot the ggems projections
fig, (axes) = plt.subplots(ncols=2, nrows=3, figsize=(5.8,10))

# Plot the denoised ggems projections
controls = pli.hyperslicer(np.array(ggems_scatter_denoised),
                           ax=axes[1][0], cmap='jet',vmax=np.percentile(np.array(ggems_scatter_denoised),99),
                           vmin=np.percentile(np.array(ggems_scatter_denoised),1),play_buttons=True)
axes[1][0].set_title('Denoised GGEMS scatter')

ggems_primary_only = np.array(ggems_primary_projections) - np.array(ggems_scatter_projections)

# Plot the ggems projections
c2 = pli.hyperslicer(np.array(ggems_primary_only), ax=axes[0][1], cmap='gray',controls=controls,
                     vmax=np.percentile(np.array(ggems_primary_only),99),
                     vmin=np.percentile(np.array(ggems_primary_only),1))
axes[0][1].set_title('GGEMS Primary')

# Plot the phantom phantom_projections
c1 = pli.hyperslicer(np.array(phantom_projections)[-1::-1], ax=axes[1][1], cmap='gray',controls=controls,
                     vmax=np.percentile(np.array(phantom_projections),99),
                     vmin=np.percentile(np.array(phantom_projections),1))
axes[1][1].set_title('Analytical Primary')

# Plot the difference between the ggems and phantom projections
perc_diff_primary = (np.array(ggems_primary_only)-np.array(phantom_projections[-1::-1]))/np.max(phantom_projections)*100

c4 = pli.hyperslicer(np.array(perc_diff_primary), ax=axes[2][1], cmap='bwr',controls=controls,
                        vmax=5,vmin=-5)
axes[2][1].set_title('Perc. Diff. [%]')

# Add a colorbar
plt.colorbar(ax=axes[2][1])

# Plot the ggems projections
c2 = pli.hyperslicer(np.array(ggems_scatter_projections), ax=axes[0][0], cmap='jet',controls=controls,
                     vmax=np.percentile(np.array(ggems_scatter_projections),99),
                     vmin=np.percentile(np.array(ggems_scatter_projections),1))

axes[0][0].set_title('GGEMS Scatter')

# Plot the ggems primary projections
c3 = pli.hyperslicer(np.array(perc_diff_small),ax=axes[2][0], cmap='bwr',vmax=5,vmin=-5,controls=controls)
plt.colorbar(ax=axes[2][0])
axes[2][0].set_title('Perc. Diff. [%]')
plt.tight_layout()

# Turn off the axes
for ax in axes.flatten():
    ax.axis('off')
# Define a function that given an angle and radius returns x and y coordinates
# def f(axis0,radius=220,offset=0,x0=256,y0=256,na=90):
#     angle = np.deg2rad(axis0*360/na)
#     x = radius * np.cos(angle+offset) + x0
#     y = radius * np.sin(angle+offset) + y0
#     return np.array([[x,y],[x,y],[x,y]])

# iplt.plot(f, ax=axes[4], a=(0.1,10), b=(0.1,10), parametric=True,ylim=[0,1],xlim=[1,na],play_buttons=True)
# plt.imshow(phantom.phantom[50, :, :], cmap='gray')
# iplt.scatter(f, ax=axes[3][0], controls=controls,play_buttons=True,c='r',marker='>',parametric=True,)

# Add a label in the image saying source position
# axes[3].text(50, 50, 'Source Position', color='r', fontsize=12)


ValueError: kwarg axis0 already exists and the new values are incompatible.

[2023-12-15 15:23:44,430] {font_manager.py:1411} DEBUG - findfont: Matching sans\-serif:style=normal:variant=normal:weight=normal:stretch=normal:size=12.0.
[2023-12-15 15:23:44,431] {font_manager.py:1423} DEBUG - findfont: score(FontEntry(fname='/home/jericho/anaconda3/envs/tigre_fastcat/lib/python3.11/site-packages/matplotlib/mpl-data/fonts/ttf/STIXSizTwoSymReg.ttf', name='STIXSizeTwoSym', style='normal', variant='normal', weight=400, stretch='normal', size='scalable')) = 10.05
[2023-12-15 15:23:44,431] {font_manager.py:1423} DEBUG - findfont: score(FontEntry(fname='/home/jericho/anaconda3/envs/tigre_fastcat/lib/python3.11/site-packages/matplotlib/mpl-data/fonts/ttf/STIXSizOneSymBol.ttf', name='STIXSizeOneSym', style='normal', variant='normal', weight=700, stretch='normal', size='scalable')) = 10.335
[2023-12-15 15:23:44,431] {font_manager.py:1423} DEBUG - findfont: score(FontEntry(fname='/home/jericho/anaconda3/envs/tigre_fastcat/lib/python3.11/site-packages/matplotlib/mpl-data/fonts

In [48]:
# Save the figure as a gif using mpl_interactions
# save as a gif
anim = controls.save_animation("phantoms_ggems_analytical+denoised.gif", fig, "axis0", interval=120)

[2023-12-07 16:51:50,649] {animation.py:1060} INFO - Animation.save using <class 'matplotlib.animation.FFMpegWriter'>
[2023-12-07 16:51:50,649] {animation.py:322} DEBUG - frame size in pixels is 580 x 964
[2023-12-07 16:51:50,650] {animation.py:338} INFO - MovieWriter._run: running command: ffmpeg -f rawvideo -vcodec rawvideo -s 580x964 -pix_fmt rgba -framerate 8.333333333333334 -i pipe: -filter_complex 'split [a][b];[a] palettegen [p];[b][p] paletteuse' -y phantoms_ggems_analytical+denoised.gif
[2023-12-07 16:51:50,655] {animation.py:366} DEBUG - MovieWriter.grab_frame: Grabbing frame.
[2023-12-07 16:51:50,748] {animation.py:366} DEBUG - MovieWriter.grab_frame: Grabbing frame.
[2023-12-07 16:51:50,836] {animation.py:366} DEBUG - MovieWriter.grab_frame: Grabbing frame.
[2023-12-07 16:51:50,927] {animation.py:366} DEBUG - MovieWriter.grab_frame: Grabbing frame.
[2023-12-07 16:51:51,015] {animation.py:366} DEBUG - MovieWriter.grab_frame: Grabbing frame.
[2023-12-07 16:51:51,105] {animati

In [24]:
# Plot the error between the ggems and phantom projections as a function of angle in terms of rmse 
rmse = []
rmse_scatter = []
for i in range(len(phantom_projections)):
    rmse_scatter.append(np.sqrt(np.mean(np.array(perc_diff_small[i])**2)))
    rmse.append(np.sqrt(np.mean(np.array(perc_diff_primary[i])**2)))

plt.figure()
plt.plot(angles,rmse_scatter,label='Scatter')
plt.plot(angles,rmse,label='Primary')
plt.xlabel('Angle [degrees]')
plt.ylabel('RMSE [%]')
plt.legend()
plt.title('RMSE vs Angle')


Text(0.5, 1.0, 'RMSE vs Angle')

In [102]:
mean_projection = np.mean(np.array(ggems_projections), axis=0)
mean_projection_zoom = zoom(downsize_block(
    mean_projection, 64), 64, order=5, mode='nearest')

pli.heatmap_slicer(np.arange(0, mean_projection.shape[0]),
                   np.arange(0, mean_projection.shape[1]),
                   (mean_projection, mean_projection_zoom),
                   cmap='jet',
                   slices='both')

(<Figure size 1800x900 with 4 Axes>,
 array([<Axes: title={'center': 'heatmap_0'}, xlabel='X', ylabel='Y'>,
        <Axes: title={'center': 'heatmap_1'}, xlabel='X'>,
        <Axes: title={'center': 'Horizontal'}>,
        <Axes: title={'center': 'Vertical'}>], dtype=object))

In [101]:
# Do the same thing but for RectBivariateSpline
from scipy.interpolate import RectBivariateSpline
import numpy as np

ggems_projections_downsized_spline = []
for ggems_projection in ggems_projections_downsized:
    # Initialize the spline
    spline = RectBivariateSpline(np.arange(ggems_projection.shape[0]), np.arange(
        ggems_projection.shape[1]), ggems_projection, kx=4, ky=4, s=2)
    # Evaluate the spline on a finer grid
    x_fine, y_fine = np.meshgrid(np.linspace(0, ggems_projection.shape[0], ggems_projection.shape[0]*4), np.linspace(
        0, ggems_projection.shape[1], ggems_projection.shape[1]*4))
    z_fine = spline.ev(x_fine, y_fine)

    ggems_projections_downsized_spline.append(z_fine)

# Plot the ggems projections
plt.figure()
controls = pli.hyperslicer(
    np.array(ggems_projections_downsized_spline), cmap='jet')

Init signature:
RectBivariateSpline(
    x,
    y,
    z,
    bbox=[None, None, None, None],
    kx=3,
    ky=3,
    s=0,
)
Docstring:     
Bivariate spline approximation over a rectangular mesh.

Can be used for both smoothing and interpolating data.

Parameters
----------
x,y : array_like
    1-D arrays of coordinates in strictly ascending order.
    Evaluated points outside the data range will be extrapolated.
z : array_like
    2-D array of data with shape (x.size,y.size).
bbox : array_like, optional
    Sequence of length 4 specifying the boundary of the rectangular
    approximation domain, which means the start and end spline knots of
    each dimension are set by these values. By default,
    ``bbox=[min(x), max(x), min(y), max(y)]``.
kx, ky : ints, optional
    Degrees of the bivariate spline. Default is 3.
s : float, optional
    Positive smoothing factor defined for estimation condition:
    ``sum((z[i]-f(x[i], y[i]))**2, axis=0) <= s`` where f is a spline
    function. Defa

In [81]:
phantom = np.load(
    '/home/jericho/Software/fastcat/fastcat/data/phantoms/ct_scan_head_mandible_large.npy')

In [84]:
plt.figure()
contr = pli.hyperslicer(phantom)

In [85]:
# load a binary file to an array
import numpy as np

phantom = np.load()

In [86]:
plt.figure()
pli.hyperslicer(phantom.phantom)

AttributeError: 'Controls' object has no attribute 'vbox'